#ライブラリインポート


In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import cv2
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time
import subprocess as sp
from datetime import datetime, timedelta, timezone
import math
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import imageio
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
vgg19 = models.vgg19(pretrained=True)
!pip install ninja
device=torch.device("cuda:0")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



     |████████████████████████████████| 112kB 6.8MB/s 


#GPU使用時間と種類を表示

In [ ]:
res = sp.Popen(["cat", "/proc/uptime"], stdout=sp.PIPE)
    # 単位はHour
use_time = float(sp.check_output(["awk", "{print $1 /60 /60 }"], stdin=res.stdout).decode().replace("\n",""))
print(use_time)

0.0666833


In [ ]:
!nvidia-smi

Thu Feb 11 11:04:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#マウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#準備

##パラメータ定義

In [ ]:
%cd drive/MyDrive/
num_thread=0
img_size=(128,96)
lr=0.0002
b1=0.5
b2=0.999
ngpu=1

/content/drive/MyDrive


##関数定義

###切り抜き

In [ ]:
def overlay(p,f,tops,pants,skirt):
  Mask=(tops+pants+skirt)
  Mask=MaskTrans(Mask)
  #print(Mask.size())
  #print(p.size())
  reverse=1-Mask
  cutout1=p*reverse
  #print(f[0][0][113][50],Mask[0][0][113][50])
  #print(Mask.size(),f[0].size())
  cutout2=Mask*f
  #print(cutout2[0][0][113][50])
  out=cutout1+cutout2
  return out

###mask閾値

In [ ]:
def  MaskTrans(mask):
  for i in range(mask.size(0)):
    for j in range(mask.size(1)):
      for k in range(mask.size(2)):
        for l in range(mask.size(3)):
          if mask[i][j][k][l]>0.6:
            mask[i][j][k][l]=1
          else:
            mask[i][j][k][l]=0
  
  return mask

###重みの初期化

In [ ]:
def init_weights(model):
  if isinstance(model.modules,nn.Conv2d):
      model.modules().weight.data.nomal_(0,0.002)
      model.modules().bias.data.zero_()
  if isinstance(model.modules,nn.ConvTranspose2d):
      model.modules().weight.data.nomal_(0,0.002)
      model.modules().bias.data.zero_()

###KLDloss


In [ ]:
def KLDloss(mu, logvar):
  lamda=0.05
  return lamda*(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()))

###Featloss


In [ ]:
def Featloss(t,f):
  L1=nn.L1Loss()
  lamda=10
  size=len(t)
  featloss = torch.full((1,), fill_value=0,dtype=torch.float32,device=device)

  for i in range(size):
    loss=L1(t[i].detach(),f[i])
    featloss+=(lamda*loss)

  return featloss

##クラス定義


###Encoder-Decoder



In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self,ngpu):
    super(EncoderDecoder,self).__init__()
    self.ngpu=ngpu
    #Encoder
    df=3#class
    self.down1_1=nn.Conv2d(3*df,32*df,3,2,1,bias=False,groups=3)
    self.innorm1_1=nn.InstanceNorm2d(32*df,eps=1e-5)

    self.Leaky=nn.LeakyReLU(0.2,inplace=True)
    self.down1_2=nn.Conv2d(32*df,64*df,3,2,1,bias=False,groups=3)
    self.innorm1_2=nn.InstanceNorm2d(64*df,eps=1e-5)

    self.down1_3=nn.Conv2d(64*df,128*df,3,2,1,bias=False,groups=3)
    self.innorm1_3=nn.InstanceNorm2d(128*df,eps=1e-5)

    self.down1_4=nn.Conv2d(128*df,256*df,3,2,1,bias=False,groups=3)
    self.innorm1_4=nn.InstanceNorm2d(256*df,eps=1e-5)

    self.gamma=nn.Conv2d(256*df,2*df,(8,6),1,0,bias=False,groups=3)
    self.beta=nn.Conv2d(256*df,2*df,(8,6),1,0,bias=False,groups=3)
    




    #Decoder
    self.down1_0=nn.Conv2d(2*df,256*df,3,1,1,groups=3)

    self.relu=nn.ReLU()
    self.leaky=nn.LeakyReLU(0.2,inplace=True)

    #2block
    self.batch2_1=nn.BatchNorm2d(256*df,eps=1e-5,affine=False)
    self.pool2_1=nn.MaxPool2d(16)
    self.share2_1=nn.Conv2d(df*1,16*df,3,1,1,groups=3)
    self.gamma2_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.beta2_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.gconv2_1=nn.Conv2d(256*df,256*df,3,1,1,groups=3)
    self.up2_1=nn.Upsample(scale_factor=2,mode="nearest")

    


    #3block
    self.batch3_1=nn.BatchNorm2d(256*df,eps=1e-5,affine=False)
    self.pool3_1=nn.MaxPool2d(8)
    self.share3_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma3_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.beta3_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.gconv3_1=nn.Conv2d(256*df,128*df,3,1,1,groups=3)
    self.up3_1=nn.Upsample(scale_factor=2,mode="nearest")

    


    #4block
    self.batch4_1=nn.BatchNorm2d(128*df,eps=1e-5,affine=False)
    self.pool4_1=nn.MaxPool2d(4)
    self.share4_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma4_1=nn.Conv2d(16*df,128*df,3,1,1,groups=3)
    self.beta4_1=nn.Conv2d(16*df,128*df,3,1,1,groups=3)
    self.gconv4_1=nn.Conv2d(128*df,64*df,3,1,1,groups=3)
    self.up4_1=nn.Upsample(scale_factor=4,mode="nearest")
    self.smooth4_1=nn.Conv2d(64*df,64*df,3,1,1,groups=3)

    

    #5block
    self.batch5_1=nn.BatchNorm2d(64*df,eps=1e-5,affine=False)
    self.pool5_1=nn.MaxPool2d(1)
    self.share5_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma5_1=nn.Conv2d(16*df,64*df,3,1,1,groups=3)
    self.beta5_1=nn.Conv2d(16*df,64*df,3,1,1,groups=3)
    self.gconv5_1=nn.Conv2d(64*df,32*df,3,1,1,groups=3)

    #+block
    self.pbatch=nn.BatchNorm2d(32*df,eps=1e-5,affine=False)
    self.ppool=nn.MaxPool2d(1)
    self.pshare=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.pgamma=nn.Conv2d(16*df,32*df,3,1,1,groups=3)
    self.pbeta=nn.Conv2d(16*df,32*df,3,1,1,groups=3)
    self.pgconv=nn.Conv2d(32*df,20*df,3,1,1,groups=3)


    #lastblock
    self.lastconv=nn.Conv2d(20*df,3,1,1,0)
    self.sigmoid=nn.Sigmoid()

    


  def Encoder(self,input): 
    #print(input.size())
    down1_1=self.down1_1(input)
    innorm1_1=self.innorm1_1(down1_1)
    #print(innorm1_1.size())

    leaky1_2=self.Leaky(innorm1_1)
    down1_2=self.down1_2(leaky1_2)
    innorm1_2=self.innorm1_2(down1_2)
    #print(innorm1_2.size())

    leaky1_3=self.Leaky(innorm1_2)
    down1_3=self.down1_3(leaky1_3)
    innorm1_3=self.innorm1_3(down1_3)
    #print(innorm1_3.size())

    leaky1_4=self.Leaky(innorm1_3)
    #print(leaky1_4.size())
    down1_4=self.down1_4(leaky1_4)
    innorm1_4=self.innorm1_4(down1_4)
    #print(innorm1_4.size())
    
    leaky1_5=self.Leaky(innorm1_4)
    #print(leaky1_5.size())
    gamma=self.gamma(leaky1_5)
    beta=self.beta(leaky1_5)
    #print(gamma.size(),beta.size())
    #print("---"*10)
    return gamma,beta




  def Decoder(self,z,seg,test=False): 
    
    #print(z.size())
    down1_0=self.down1_0(z)
    #print(down1_0.size())


    #2block
    batch2_1=self.batch2_1(down1_0)
    share2_1=self.relu(self.share2_1(self.pool2_1(seg)))
    gamma2_1=self.gamma2_1(share2_1)
    beta2_1=self.beta2_1(share2_1)
    temp2_1=(batch2_1*gamma2_1)+beta2_1
    out2_1=self.up2_1(self.gconv2_1(self.leaky(temp2_1)))
    #print(out2_1.size())


    #3block
    batch3_1=self.batch3_1(out2_1)
    share3_1=self.relu(self.share3_1(self.pool3_1(seg)))
    gamma3_1=self.gamma3_1(share3_1)
    beta3_1=self.beta3_1(share3_1)
    #print(batch3_1.size(),gamma3_1.size())
    temp3_1=(batch3_1*gamma3_1)+beta3_1
    out3_1=self.up3_1(self.gconv3_1(self.leaky(temp3_1)))
    #print(out3_1.size())


    #4block
    batch4_1=self.batch4_1(out3_1)
    share4_1=self.relu(self.share4_1(self.pool4_1(seg)))
    gamma4_1=self.gamma4_1(share4_1)
    beta4_1=self.beta4_1(share4_1)
    #print(batch4_1.size(),gamma4_1.size())
    temp4_1=(batch4_1*gamma4_1)+beta4_1
    out4_1=self.up4_1(self.gconv4_1(self.leaky(temp4_1)))
    out4_1=self.smooth4_1(out4_1)
    #print(out4_1.size())

    
    #5block
    batch5_1=self.batch5_1(out4_1)
    share5_1=self.relu(self.share5_1(self.pool5_1(seg)))
    gamma5_1=self.gamma5_1(share5_1)
    beta5_1=self.beta5_1(share5_1)
    #print(batch5_1.size(),gamma5_1.size())
    temp5_1=(batch5_1*gamma5_1)+beta5_1
    #for i in range(32):
      #vutils.save_image(batch5_1[2,i,:,:],f"batch5_1_{i}.png")
    #for i in range(32):
      #vutils.save_image(gamma5_1[2,i,:,:],f"gamma5_1_{i}.png")
    #for i in range(32):
      #vutils.save_image(beta5_1[2,i,:,:],f"beta5_1_{i}.png")
    out5_1=self.gconv5_1(self.leaky(temp5_1))
    #print(out5_1.size())

    #+block
    batch=self.pbatch(out5_1)
    share=self.relu(self.pshare(self.ppool(seg)))
    gamma=self.pgamma(share)
    beta=self.pbeta(share)
    temp=(batch*gamma)+beta
    out=self.pgconv(self.leaky(temp))
    #print(out.size())
    last=self.leaky(out)   
    #for i in range(20):
      #vutils.save_image(last[2,i,:,:],f"last{i}.png")
    out5=self.sigmoid(self.lastconv(last))

    return out5,last


  def reparameterize(self,mu, logvar):
      std = torch.exp(0.5 * logvar) #分散共分散のlogvarを標準偏差stdに変換
      #print(std.size())
      eps = torch.randn_like(std)/4
      #print(mu)
      #print(std,eps)
      #print(eps.mul(std) + mu)
      return eps.mul(std) + mu


  def forward(self,input,seg,Z,test=False,pre=False,encodeonly=False):
    if not test and not pre and not encodeonly:
      mu,logvar=self.Encoder(input)
      z=self.reparameterize(mu,logvar)
      #print(input.size(),seg.size())
      fake,last=self.Decoder(z,seg)
      return fake,mu,logvar,z,last
    
    elif pre==True:
      fake,_=self.Decoder(Z,seg,test=True)
      return fake

    elif encodeonly==True:
      mu,logvar=self.Encoder(input)
      z=self.reparameterize(mu,logvar)
      return z,mu,logvar

    else:
      z=torch.randn((1,6,1,1)).to(device)
      z=z.repeat(seg.size(0),1,1,1)
      
      fake,_=self.Decoder(z,seg,test=True)
      return fake,z

###Discriminator

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,ngpu):    
    super(Discriminator,self).__init__()
    self.ngpu=ngpu
    df=3
    dc=df+3#class
    #1block
    self.pool=nn.AvgPool2d(2)
    self.down1_1=nn.Conv2d(dc,8*dc,4,2,2)
    self.leaky=nn.LeakyReLU(0.2)

    #2block
    self.down1_2=nn.Conv2d(8*dc,16*dc,4,2,2)
    self.innorm1_2=nn.InstanceNorm2d(16*dc)

    #3block
    self.down1_3=nn.Conv2d(16*dc,32*dc,4,2,2)
    self.innorm1_3=nn.InstanceNorm2d(32*dc)

    #4block
    self.down1_4=nn.Conv2d(32*dc,64*dc,4,1,2)
    self.innorm1_4=nn.InstanceNorm2d(64*dc)

    #5block
    self.down1_5=nn.Conv2d(64*dc,1,4,1,2)
    self.sigmoid=nn.Sigmoid()

  def Discriminator(self,image,seg): 
    #print(image.size(),seg.size())
    featmap=[]
    data=torch.cat([image,seg],dim=1)
    
    #1block
    pool1_1=self.pool(data)
    down1_1=self.down1_1(pool1_1)
    leaky1_1=self.leaky(down1_1)
    featmap.append(leaky1_1)
    #print(leaky1_1.size())

    #2block
    down1_2=self.down1_2(leaky1_1)
    innorm1_2=self.innorm1_2(down1_2)
    leaky1_2=self.leaky(innorm1_2)
    featmap.append(leaky1_2)
    #print(leaky1_2.size())

    #3block
    down1_3=self.down1_3(leaky1_2)
    innorm1_3=self.innorm1_3(down1_3)
    leaky1_3=self.leaky(innorm1_3)
    featmap.append(leaky1_3)
    #print(leaky1_3.size())



    #4block
    down1_4=self.down1_4(leaky1_3)
    innorm1_4=self.innorm1_4(down1_4)
    leaky1_4=self.leaky(innorm1_4)
    featmap.append(leaky1_4)
    #print(leaky1_4.size())

    down1_5=self.down1_5(leaky1_4)
    #print(down1_5.size())
    #print("---"*10)

    return self.sigmoid(down1_5),featmap

###知覚損失



In [ ]:
class Vgg19Loss(nn.Module):
    def __init__(self):
        super(Vgg19Loss, self).__init__()
        features1=list(vgg19.features)[:3]
        features2=list(vgg19.features)[:8]
        features3=list(vgg19.features)[:13]
        features4=list(vgg19.features)[:22]
        features5=list(vgg19.features)[:31]
        self.features1=nn.ModuleList(features1).eval()
        self.features2=nn.ModuleList(features2).eval()
        self.features3=nn.ModuleList(features3).eval()
        self.features4=nn.ModuleList(features4).eval()
        self.features5=nn.ModuleList(features5).eval()

    def forward(self,x,y):
        t1=x
        t2=y
        loss1=nn.MSELoss()
        loss2=nn.MSELoss()
        loss3=nn.MSELoss()
        loss4=nn.MSELoss()
        loss5=nn.MSELoss()

        for f in self.features1:
            x=f(x)
            y=f(y)
        #print(x,y)
        f1loss=torch.sqrt(loss1(x,y)/(64*128*96))
    
        
        x=t1
        y=t2
        for f in self.features2:
            x=f(x)
            y=f(y)
       # print(x.size(),y.size())
        f2loss=torch.sqrt(loss2(x,y)/(128*64*48))

        x=t1
        y=t2
        for f in self.features3:
            x=f(x)
            y=f(y)
       # print(x.size(),y.size())
        f3loss=torch.sqrt(loss3(x,y)/(256*32*24))

        x=t1
        y=t2
        for f in self.features4:
            x=f(x)
            y=f(y)
        #print(x.size(),y.size())
        f4loss=torch.sqrt(loss4(x,y)/(512*16*12))

        x=t1
        y=t2
        for f in self.features5:
            x=f(x)
            y=f(y)
        #print(x.size(),y.size())
        f5loss=torch.sqrt(loss5(x,y)/(512*8*6))
        lamda=10
        #print((f1loss+f2loss+f3loss+f4loss+f5loss))

        #return (f1loss+f2loss+f3loss+f4loss+f5loss)*10
        return (f1loss+f2loss+f3loss+f4loss+f5loss)*10

##Encoder-Decoder,Discriminatorの初期化

In [ ]:
netG=EncoderDecoder(ngpu).to(device)
netG.apply(init_weights)

netD=Discriminator(ngpu).to(device)
netD.apply(init_weights)

Discriminator(
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (down1_1): Conv2d(6, 48, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
  (leaky): LeakyReLU(negative_slope=0.2)
  (down1_2): Conv2d(48, 96, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
  (innorm1_2): InstanceNorm2d(96, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (down1_3): Conv2d(96, 192, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
  (innorm1_3): InstanceNorm2d(192, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (down1_4): Conv2d(192, 384, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (innorm1_4): InstanceNorm2d(384, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (down1_5): Conv2d(384, 1, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (sigmoid): Sigmoid()
)

##Adam最適化＆損失関数

In [ ]:
L1=nn.L1Loss()
Lper=Vgg19Loss().to(device)
Ladv=nn.BCELoss()
tmapL1=nn.L1Loss()
smapL1=nn.L1Loss()
pmapL1=nn.L1Loss()

real_label=1
fake_label=0

optimizernetG=optim.Adam(netG.parameters(),lr=lr,betas=(b1,b2))
optimizernetD=optim.Adam(netD.parameters(),lr=lr,betas=(b1,b2))

#ロード

In [ ]:
!unzip DiversityDesign_data.zip

In [ ]:
!unzip segmentation.zip

In [ ]:
%cd ..
%cd ..

/content/drive
/content


In [ ]:
PATHG='drive/My Drive/DiversityDesign_data/model/Encoder_Decoder.pth'
#Unet = EncoderDecoder(ngpu).to(device)
optimizernetG= optim.Adam(netG.parameters(),lr=lr,betas=(b1,b2))
Ladv=nn.BCELoss()
L1=nn.L1Loss()
Lper=Vgg19Loss().to(device)

checkpoint = torch.load(PATHG)
netG.load_state_dict(checkpoint['model_state_dict'])
optimizernetG.load_state_dict(checkpoint['optimizer_state_dict'])
Ladv.load_state_dict(checkpoint['criterion1'])
L1.load_state_dict(checkpoint['criterion2'])
Lper.load_state_dict(checkpoint['criterion3'])
epoch = checkpoint['epoch']
#img_list=checkpoint['image']
Sum_losses=checkpoint['Sum_losses']
Generator_losses=checkpoint['Generator']
kld_losses=checkpoint['kld']
vgg_losses=checkpoint['Vgg']
L1_losses=checkpoint["L1"]
judge1=checkpoint['judge1']
judge2=checkpoint['judge2']
judge3=checkpoint['judge3']

#システム


##前処理

In [ ]:
!mkdir target
!mkdir out
!mkdir person
!mkdir tops
!mkdir pants
!mkdir skirt
!mkdir result
!mkdir person/person
!mkdir tops/mask
!mkdir pants/mask
!mkdir skirt/mask

**以下のコードを実行する前に、フォルダ"target"に人物画像を"0.png"として配置してください**

In [ ]:
!python drive/MyDrive/segmentation/simple_extractor.py --dataset lip --model-restore drive/MyDrive/segmentation/exp-schp-201908261155-lip.pth --input-dir target --output-dir out

In [ ]:
name=0
img1=cv2.imread(f"out/{name}.png")
img2=cv2.imread(f"target/{name}.png")
if img1.shape[0]>img1.shape[1]:
    temp=int(img1.shape[0]/128) #[1]/96
    w=temp*96
    h=temp*128
    if h>img1.shape[0]:
      w=w/2
      h=h/2

    y2=img1.shape[0]-(img1.shape[0]-h)/2
    y1=img1.shape[0]-((img1.shape[0]-h)/2)-h
    if (y1-round(y1))==0.5:
      y1=y1-0.5
      y2=y2+0.5

    x2=img1.shape[1]-(img1.shape[1]-w)/2
    x1=img1.shape[1]-((img1.shape[1]-w)/2)-w
    if (x1-round(x1))==0.5:
      x1=x1+0.5
      x2=x2+0.5
    img1=img1[int(y1):int(y2),int(x1):int(x2),:]
    img2=img2[int(y1):int(y2),int(x1):int(x2),:]
    
    
    img1=cv2.resize(img1,(96,128))
    img2=cv2.resize(img2,(96,128))

elif img1.shape[0]<img1.shape[1]:
    temp=int(img1.shape[0]/128)
    w=temp*96
    h=temp*128
    if w>img1.shape[1]:
      w=w/2
      h=h/2

    y2=img1.shape[0]-(img1.shape[0]-h)/2
    y1=img1.shape[0]-((img1.shape[0]-h)/2)-h
    if (y1-round(y1))==0.5:
      y1=y1-0.5
      y2=y2+0.5

    x2=img1.shape[1]-(img1.shape[1]-w)/2
    x1=img1.shape[1]-((img1.shape[1]-w)/2)-w
    if (x1-round(x1))==0.5:
      x1=x1+0.5
      x2=x2+0.5
    img1=img1[int(y1):int(y2),int(x1):int(x2),:]
    img2=img2[int(y1):int(y2),int(x1):int(x2),:]
    #print(img1.shape)
   
    img1=cv2.resize(img1,(96,128))
    img2=cv2.resize(img2,(96,128))

cv2_imshow(img1)
cv2.imwrite("1.jpg",img1)
cv2.imwrite(f"person/person/{name}.jpg",img2)



img1=img1.transpose(2,0,1)
np.set_printoptions(threshold=np.inf)
zero=np.zeros((1,img1.shape[1],img1.shape[2]))
for i in range(img1.shape[1]):
    for j in range(img1.shape[2]):
      if img1[0][i][j]==128 and img1[1][i][j]==0 and img1[2][i][j]==128:
        zero[0][i][j]=1
  
mask=zero.transpose(1,2,0)
cv2.imwrite(f"tops/mask/{name}.jpg",mask*255)

zero=np.zeros((1,img1.shape[1],img1.shape[2]))
for i in range(img1.shape[1]):
    for j in range(img1.shape[2]):
      if img1[0][i][j]==0 and img1[1][i][j]==0 and img1[2][i][j]==192:
        zero[0][i][j]=1
  
mask=zero.transpose(1,2,0)
cv2.imwrite(f"pants/mask/{name}.jpg",mask*255)

zero=np.zeros((1,img1.shape[1],img1.shape[2]))
for i in range(img1.shape[1]):
    for j in range(img1.shape[2]):
      if img1[0][i][j]==128 and img1[1][i][j]==0 and img1[2][i][j]==64:
        zero[0][i][j]=1
  
mask=zero.transpose(1,2,0)
cv2.imwrite(f"skirt/mask/{name}.jpg",mask*255)

**セグメンテーションマップが得られない場合は、フォルダ"target"を消去して別の画像で再度"0.png"として実行してください**

##テスト

In [ ]:
dataroot4="pants"
dataroot8="skirt"
dataroot10="tops"
dataroot11="person"
num_thread=0
batch_size=10
num_epoch=15
img_size=(128,96)
lr=0.0002
b1=0.5
b2=0.999
ngpu=1
test_pants_mask_dataset=dset.ImageFolder(root=dataroot4,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_skirt_mask_dataset=dset.ImageFolder(root=dataroot8,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_tops_mask_dataset=dset.ImageFolder(root=dataroot10,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_person_dataset=dset.ImageFolder(root=dataroot11,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))

test_pants_mask_dataloader=torch.utils.data.DataLoader(test_pants_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_skirt_mask_dataloader=torch.utils.data.DataLoader(test_skirt_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_tops_mask_dataloader=torch.utils.data.DataLoader(test_tops_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_person_dataloader=torch.utils.data.DataLoader(test_person_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)

#device=torch.device("cuda:0")
print(len(test_person_dataset))

In [ ]:
n=100
netG.eval()
z=torch.randn(1,6,1,1).to(device)
tops=False #topsの変化有無
pants=True #pantsの変化有無
skirt=False #skirtの変化有無
for epoch in range(n):
   
   iter_person=iter(test_person_dataloader)

   iter_tops_mask = iter(test_tops_mask_dataloader)
   iter_pants_mask=iter(test_pants_mask_dataloader)
   iter_skirt_mask= iter(test_skirt_mask_dataloader)

   t1=time.time()
   for tops_data in test_tops_mask_dataloader:
        #0.バッチデータの取得     
        
        person_real_batch=next(iter_person)
        tops_mask_real_batch=next(iter_tops_mask)
        pants_mask_real_batch=next(iter_pants_mask)
        skirt_mask_real_batch=next(iter_skirt_mask)
       
        person=person_real_batch[0].to(device)
        tops_mask=tops_mask_real_batch[0].to(device)              #[][][][]
        pants_mask=pants_mask_real_batch[0].to(device)      #[][][][]
        skirt_mask=skirt_mask_real_batch[0].to(device)  

        b_size = tops_mask.size(0)#バッチサイズを計算
        #    
        #データの連結
        seg=torch.cat([tops_mask,pants_mask,skirt_mask],dim=1)
        
        #fakeを生成
        if tops:
          z[0,0,:,:]=torch.randn(1,1,1,1)
          z[0,1,:,:]=torch.randn(1,1,1,1)
        if pants:
          z[0,2,:,:]=torch.randn(1,1,1,1)
          z[0,3,:,:]=torch.randn(1,1,1,1)
        if skirt:
          z[0,4,:,:]=torch.randn(1,1,1,1)
          z[0,5,:,:]=torch.randn(1,1,1,1)

        fake=netG(_,seg,z,pre=True)
        fake_image=overlay(person,fake[0],tops_mask,pants_mask,skirt_mask)
   vutils.save_image(fake_image[0],f"result/{name}_{epoch}.jpg")
   if (epoch+1)%5==0:
    print(epoch+1)

images=[]
for i in range(n):
  images.append(imageio.imread(f"result/{name}_{i}.jpg"))
imageio.mimsave(f"{name}.gif",images,duration=0.35)

**"0.gif"に多様な色の試着画像が保存されます**

##削除

In [ ]:
!rm "0.gif"
%cd result/
!rm *
%cd .. 
!rm "out/0.png"
!rm "pants/mask/0.jpg"
!rm "person/person/0.jpg"
!rm "skirt/mask/0.jpg"
!rm "tops/mask/0.jpg"
!rm "target/0.png"